In [1]:
import sys 
sys.path.insert(0,'/home/mohit/Downloads/code_notebooks/deep_boltzmann')
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import keras
import tensorflow as tf

Using TensorFlow backend.


In [2]:
from sklearn.metrics import mean_squared_error
from keras import backend as K
from keras import *
from keras.layers import *

In [3]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import ast

In [4]:
from deep_boltzmann.models import ParticleDimer
from deep_boltzmann.networks.invertible import invnet, EnergyInvNet, create_RealNVPNet
from deep_boltzmann.sampling import GaussianPriorMCMC
from deep_boltzmann.sampling.latent_sampling import BiasedModel
from deep_boltzmann.sampling.permutation import HungarianMapper
from deep_boltzmann.util import load_obj, save_obj

In [5]:
from deep_boltzmann.sampling.analysis import free_energy_bootstrap, mean_finite, std_finite

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda

In [7]:
# Switch AUTORELOAD ON. Disable this when in production mode!
%load_ext autoreload
%autoreload 2

In [8]:
trajdict = np.load('output.npz')
traj_closed_train = trajdict['traj_closed_train_hungarian']
traj_open_train = trajdict['traj_open_train_hungarian']
traj_closed_test = trajdict['traj_closed_test_hungarian']
traj_open_test = trajdict['traj_open_test_hungarian']
x = np.vstack([traj_closed_train, traj_open_train])
xval = np.vstack([traj_closed_test, traj_open_test])

In [38]:
from tensorflow import set_random_seed
set_random_seed(5)

In [9]:
Pmodel = ParticleDimer()

In [10]:
print(Pmodel.energy(traj_open_train[85000:95000]))

[53.1052475  53.1052475  53.1052475  ... 47.43473434 47.43473434
 47.26223755]


In [11]:
print(traj_open_train[5][:])

[-0.7737231   0.0229922   0.7693601  -0.0248092  -3.0438566  -2.9522533
 -3.0273473  -1.8176557  -3.0248756  -0.6278732  -2.968509    0.5933634
 -3.05296     1.794715   -3.051158    2.9943008  -1.7285206  -3.0098464
 -1.8069663  -1.7470042  -1.7986456  -0.53872395 -1.7701639   0.5886228
 -1.7998496   1.8089703  -1.8354506   2.9848778  -0.55377567 -2.989971
 -0.59609133 -1.8446153  -0.5866792  -0.6150976  -0.6053329   0.68161553
 -0.5770965   1.8387243  -0.60820425  3.015796    0.6113206  -3.0218046
  0.5989113  -1.852775    0.5690953  -0.62073636  0.5842592   0.57050437
  0.5738882   1.7786775   0.5548503   2.9658859   1.8295407  -3.0401685
  1.7996424  -1.7844839   1.8110508  -0.59060943  1.7638934   0.6411102
  1.8230119   1.8495789   1.7729679   2.9974556   2.9385495  -3.0330408
  3.0325608  -1.8080686   3.0145466  -0.6335853   2.9596307   0.60284555
  2.9931705   1.7696677   2.9961472   3.056316  ]


In [39]:
MLmodel = Sequential()

In [40]:
MLmodel.add(Dense(128, activation='tanh', input_dim=4,kernel_initializer='random_normal', bias_initializer='random_uniform' ))
MLmodel.add(Dense(128, activation='relu', kernel_initializer='random_normal', bias_initializer='zeros'))
MLmodel.add(Dense(128, activation='relu',kernel_initializer='random_normal', bias_initializer='random_uniform'))

In [41]:
om = keras.optimizers.Adam()

In [42]:
MLmodel.add(Dense(76, activation = 'sigmoid', kernel_initializer='random_normal', bias_initializer='random_uniform'))

In [43]:
def energyLoss(y_true, y_pred):
    #end  = (ParticleDimer().energy_tf(y_true) - ParticleDimer().energy_tf(y_pred) )**2
    from deep_boltzmann.util import linlogcut
    temp = Lambda(lambda x:  x - 0.5)(y_pred)
    temp = temp *5.5
    #return linlogcut(ParticleDimer().energy_tf(temp),10**4, 10**20, tf = True)
    return K.sqrt(K.mean(K.square(ParticleDimer().energy_tf(y_true) - linlogcut(ParticleDimer().energy_tf(temp),10**4, 10**20, tf = True) ) ))
    return end
    #return mean_squared_error(ParticleDimer().energy(y_true)[:,], ParticleDimer().energy(y_pred)[:,] )

In [44]:
MLmodel.compile(optimizer = om, loss = energyLoss, metrics= [energyLoss])

In [46]:
MLmodel.fit(x= np.array( traj_open_train[5000:55000,:4] ) , y = np.array( traj_open_train[5000:55000] ) ,epochs = 20, batch_size = 512)

Epoch 1/20
50000/50000 [==============================] - 4s 72us/step - loss: 9981.1278 - energyLoss: 9981.1278
Epoch 2/20
50000/50000 [==============================] - 4s 74us/step - loss: 9981.1278 - energyLoss: 9981.1278
Epoch 3/20
50000/50000 [==============================] - 4s 74us/step - loss: 9981.1278 - energyLoss: 9981.1278
Epoch 4/20
50000/50000 [==============================] - 4s 74us/step - loss: 9981.1277 - energyLoss: 9981.1277
Epoch 5/20
50000/50000 [==============================] - 3s 67us/step - loss: 9981.1278 - energyLoss: 9981.1278
Epoch 6/20
50000/50000 [==============================] - 3s 60us/step - loss: 9981.1279 - energyLoss: 9981.1278
Epoch 7/20
50000/50000 [==============================] - 3s 60us/step - loss: 9981.1278 - energyLoss: 9981.1278
Epoch 8/20
50000/50000 [==============================] - 3s 60us/step - loss: 9981.1278 - energyLoss: 9981.1278
Epoch 9/20
50000/50000 [==============================] - 3s 60us/step - loss: 9981.1278 - energ

In [ ]:
y_pred = MLmodel.predict(x= traj_open_train[60000:70000,:4])

In [51]:
for layer in MLmodel.layers:
    print(layer.get_config())

{'name': 'dense_9', 'trainable': True, 'batch_input_shape': (None, 4), 'dtype': 'float32', 'units': 128, 'activation': 'tanh', 'use_bias': True, 'kernel_initializer': {'class_name': 'RandomNormal', 'config': {'mean': 0.0, 'stddev': 0.05, 'seed': None}}, 'bias_initializer': {'class_name': 'RandomUniform', 'config': {'minval': -0.05, 'maxval': 0.05, 'seed': None}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
{'name': 'dense_10', 'trainable': True, 'units': 128, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'RandomNormal', 'config': {'mean': 0.0, 'stddev': 0.05, 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
{'name': 'dense_11', 'trainable': True, 'units': 128, 'activation': 'relu', 'use_bias': True, 'kerne

In [22]:
temp = y_pred - 0.5
temp = temp *5.0
print(temp[10])

[ 5.5217147e-02 -1.7952472e-02  2.5000000e+00  2.5000000e+00
 -2.8500259e-02  4.2590201e-02 -4.7314912e-02  2.2207201e-02
 -5.2977502e-02 -5.9573501e-02 -5.0252825e-02  4.0838420e-02
  2.7154088e-02 -1.0217875e-02  1.0291338e-02 -9.5643103e-03
 -5.2872896e-02  4.3327808e-02  2.8044283e-02  7.4324012e-03
  1.6116500e-02  1.0500550e-02 -8.9479983e-03 -5.5612773e-02
  5.6721270e-02  5.7006180e-02 -2.9437542e-02 -5.8585852e-02
  4.5840740e-02  3.6917329e-02  4.7627389e-02 -5.0990283e-02
  3.3496618e-02 -7.6787174e-03  9.3656778e-03  5.8981478e-02
  1.6210973e-02  1.6610622e-02  3.6094487e-02 -2.4378151e-02
  4.5169294e-02  1.3378263e-03  4.0813088e-02  1.7218590e-02
 -3.7708730e-02  1.4799237e-02 -2.5760084e-02  6.3097477e-03
 -3.8938969e-02  5.5022538e-02 -5.9999526e-02  4.4349432e-02
 -3.5337657e-02  1.6500652e-02  6.1662495e-02 -4.1292906e-02
  2.5159419e-02 -4.2453557e-02 -5.4469258e-02 -1.2807995e-02
  4.2313337e-03  5.5913627e-02  2.5000000e+00  2.5000000e+00
 -1.8837750e-02  1.39263

In [28]:
dis= []
mn = 100000.0
for i in range(0,76,2):
    d = []
    for j in range(i+2,76,2):
        d.append( np.linalg.norm(temp[10 , i:i+2] - temp[10, j:j+2]) )
        if(np.linalg.norm(temp[40 , i:i+2] - temp[40, j:j+2])  == 0.0):
            print(i,j)
        mn = min(mn, np.linalg.norm(temp[10 , i:i+2] - temp[10, j:j+2]) )
    dis.append(d)
print(mn)
print(dis)

2 62
0.0
[[3.509565, 0.103315145, 0.11011641, 0.11592408, 0.12074885, 0.029109435, 0.04570218, 0.124252684, 0.03718543, 0.048357368, 0.0744007, 0.07497374, 0.093901485, 0.05566518, 0.0338984, 0.024027722, 0.0895611, 0.05211611, 0.020173386, 0.021750286, 0.03800632, 0.09852864, 0.084533826, 0.11912484, 0.13098247, 0.096887514, 0.024214013, 0.038778476, 0.10980698, 0.08975385, 3.509565, 0.08062497, 0.07682309, 0.12011283, 0.06660691, 0.048348714, 0.09527228], [3.5259292, 3.5536277, 3.6151226, 3.5427766, 3.5236573, 3.5350478, 3.5429366, 3.5104783, 3.516715, 3.5813375, 3.4551165, 3.5978348, 3.4770207, 3.538599, 3.5173984, 3.4873817, 3.5123255, 3.5275085, 3.5027854, 3.4945393, 3.5519276, 3.5493598, 3.5247874, 3.5473676, 3.549043, 3.521883, 3.5480847, 3.5832274, 3.4931962, 0.0, 3.5390828, 3.5745595, 3.5236957, 3.5233717, 3.492381, 3.5403638], [0.027739104, 0.10505503, 0.021822989, 0.076720916, 0.06499909, 0.024383796, 0.066583455, 0.054958172, 0.10013049, 0.086432226, 0.1011804, 0.074557126,

In [21]:
print(y_pred[1])

[0.5110434  0.4964095  1.         1.         0.49429995 0.50851804
 0.49053702 0.50444144 0.4894045  0.4880853  0.48994943 0.5081677
 0.5054308  0.49795642 0.50205827 0.49808714 0.48942542 0.50866556
 0.50560886 0.5014865  0.5032233  0.5021001  0.4982104  0.48887745
 0.51134425 0.51140124 0.4941125  0.48828283 0.50916815 0.50738347
 0.5095255  0.48980194 0.5066993  0.49846426 0.50187314 0.5117963
 0.5032422  0.5033221  0.5072189  0.49512437 0.50903386 0.50026757
 0.5081626  0.5034437  0.49245825 0.50295985 0.49484798 0.50126195
 0.4922122  0.5110045  0.4880001  0.5088699  0.49293247 0.50330013
 0.5123325  0.49174142 0.5050319  0.4915093  0.48910615 0.4974384
 0.50084627 0.5111827  1.         1.         0.49623245 0.50278527
 0.49609053 0.49287647 0.4922098  0.51132184 0.4998363  0.50361085
 0.5070095  0.50519764 0.4939216  0.50477105]


In [22]:
print(ParticleDimer().energy(y_pred))

[inf inf inf ... inf inf inf]


/home/mohit/Downloads/code_notebooks/deep_boltzmann/deep_boltzmann/models/particle_dimer.py:135: RuntimeWarning: overflow encountered in power
  E = 0.5*self.params['eps']*np.sum(D2rel**6, axis=(1, 2))  # do 1/2 because we have double-counted each interaction
/home/mohit/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [23]:
print(y_pred[0])

[0.5055054  0.5040377  0.4953292  0.5056766  0.5078302  0.49619603
 0.515007   0.4944263  0.50365126 0.4908791  0.49212477 0.50960076
 0.5004689  0.49366772 0.48370853 0.4944977  0.51133174 0.5157838
 0.50621367 0.487342   0.49562448 0.48559222 0.50630856 0.50944406
 0.502216   0.50735    0.504218   0.49577746 0.5124366  0.5034814
 0.5021055  0.49739888 0.5016339  0.49918035 0.5091312  0.4991595
 0.49505496 0.49379987 0.5176669  0.50181985 0.5021656  0.5071673
 0.4894404  0.50203085 0.48399302 0.4992142  0.49719873 0.49074093
 0.4980657  0.5058365  0.49007043 0.5073575  0.5051535  0.49591243
 0.5079856  0.5013233  0.48463264 0.49162614 0.504262   0.50219667
 0.5085697  0.5080575  0.49288863 0.50114834 0.4867405  0.49277544
 0.51573586 0.49303478 0.49390447 0.50383383 0.5077228  0.49721968
 0.48501158 0.50921553 0.49333295 0.48505056]
